<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# Webscraping with BeautifulSoup and requests

_Authors: Riley Daggle & Jeff Hale_

---

## Learning Objectives

After this lesson students will be able to:
- Get HTML content from websites with requests 
- Parse website content with BeautifulSoup


### Prior knowledge required
- Python and pandas basics
---

# Web scraping issues

## Terms of service ⭐️
Google is your friend. See what it says about webscraping.

The law is unresolved, but generally, if the data is publicly available and you are using it for educational purposes, it's unlikely that you will have problems. 

![](./assets/scraping-legal-info.png)

[Source](https://mccarthygarberlaw.com/a-comprehensive-legal-guide-to-web-scraping-in-the-us/)

### robots.txt 🤖

https:my_site_name_here.com/robots.txt tells you what pages the site would like scrapers/crawlers to scrape/crawl. 

Read more [here](https://www.promptcloud.com/blog/how-to-read-and-respect-robots-file/#:~:text=txt%20file%20of%20a%20website%20you're%20trying%20to%20crawl,site%20are%20crawlable%20by%20bots.&text=You%20should%20steer%20clear%20from,txt.).

---

## Let's do some scraping
### Imports

In [ ]:
# install if needed
# pip install bs4

In [ ]:


# import pandas, bs4, and requests

#### Use the requests library to get the content of a sample webpage

In [ ]:
bs4.__version__

In [ ]:
url = 'https://rldaggie.github.io/sample-html/'


#### What did we get back?

#### Our response object has a lot more in it, we just have to get it out.
#### Status Codes

## Status codes
Status codes tell you how the target server responded to your request

#### 200 = OK

#### 300s = Redirection

#### 400s = Client Error
- 400 = Bad Request
- 403 = Forbidden (not authorized)
- 404 = Not Found

#### 500s = Server Error

If your request was successful, you now have the contents of the webpage stored in memory on your machine.

---
#### Let's get the good stuff 🚀

#### We could parse this by hand 😿

#### But that would be painful and we can instead use a library 😀
### Create a `BeautifulSoup` object

### What is it

In [ ]:
type(soup)

#### Let's take a look at it

# `soup.find()`

### Returns either:

1. A soup object of the first match
2. `None`

#### Get the text in the tag

#### Get the attributes of the tag

# ⭐️ ⭐️`soup.find_all()` ⭐️ ⭐️

### Returns a **_LIST_** (techically a bs4.element.ResultSet) of soup objects that match your query.

## Behaves differently than `find()`

#### Make a list comprehension that creates a list containing only the text of the tags

#### List comprehension that puts the classes of the h1 tags in a list

## Todo List

Find the ordered list items where the class = 'done'

#### Get the list item texts from the ol

## Let's scrape a beer reviews website

### TOS

Find the Terms of Service for the website. 

### robots.txt

- robots.txt https:my_site_name_here.com/robots.txt tells you what pages it would like you to crawl.

#### Get the content

#### Find the content of any H2 tags with BS4

#### Grab all the Trending Beers 

## More Issues
Sometimes the HTML doesn't appear right away. Maybe you need to simulate clicking on buttons.

You can use a headless browser. 

- Selenium with Chromium will do the job. Here's an article on the topic: https://www.scrapingbee.com/blog/selenium-python/

- [Scrapy](https://scrapy.org/) is another option for scraping websites. It makes requests and gets data but is more powerful and complex than requests with BS4.

- Your IP address (or username if logged in) can get blocked if you are deemed to be malicious. 

- DOS (Denial of Service) attacks are real and if you ping a website lots and lots of times quickly you might get blocked, regardless of what robots.txt or the terms of use say.

- If you want to scrape repeatedly, make sure the website doesn't get changed andeaking how you grab the data!

## Summary

You've seen how to use requests with BS4 to get HTML and parse it.

Scraping websites is brittle and can be frustrating. But it's pretty cool. 😉

### Check for understanding

- What requests method do you use to grab HTML?
- How do you get HTML content out of the requests object?